# 1. In optimisation loop



# 1.1. Merit functions in spectral optimisation

# 1.1.1. Abudance map unmxing

CA

In [1]:
from sklearn.metrics import accuracy_score

def classification_score (a_map, pred_map):
    N,M,P = a_map.shape

    a_map = a_map.reshape((N*M,P))
    
    score = zeros((N*M))
    S = zeros((P))
    pred_map = pred_map.reshape((N*M,P))
    y_true = zeros((N*M))
    y_pred = zeros((N*M))

    
    for l in range (P):
        nor = max(a_map[:,l])
        nor2 = min(a_map[:,l])
        c = 0.5*(nor+nor2)
        
        for k in range (N*M):
            
            y_true[k] = a_map[k,l]
            y_pred[k] = pred_map[k,l]
            y_true[k] = (y_true[k] > c)
            y_pred[k] = (y_pred[k] > c)
        S[l] = accuracy_score(y_true,y_pred)
    
    return S

def compute_classification_acc(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q
            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)
    
    predicted = predicted.reshape(N,M,P)

    accuracy = classification_score (a_map, predicted)
    
    accuracy = -mean(accuracy)
    
    if not accuracy or isnan(accuracy):
        return 100.
    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

In [2]:
from scipy.stats import entropy

def compute_unmixing_accuracy_entropy_first_option(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q
            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)

    # Take entropy between true abundance map and predicted abundance map.
    x = predicted
    y = abundance_map.reshape((N*M,len(endmember_sig)))
    difference =  numpy.absolute((x - y))
    
    #assume the difference followes the guassian distributioin
    #standard div = 0.5, this is a parameter change with dataset, 0.5 for the simple gussian dataset
    #/. not a good assumption, should think it later
    
    sigma = 0.5
    p = exp(-(difference)**2/(2*sigma**2))

    acc = entropy([p,1-p], base=2)
    
    accuracy = acc.sum(axis=0)
    accuracy = accuracy.sum()

    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

In [3]:
def intensity_recieved(hypercube,opti_bw, opti_wavelength,wavelength, verbose=False):
    
    '''
    Inputs:
        1) hypercube (N X M x Q) - 3D synthetic hypercube.
        2) bandwidth
        3) wavelength
        4) number of block (mxXny)
        5) number of filter 
    
    
        ranking depends on three factor: 
        1) Total area of each filter 
        2) The total intensity recieved by this chanel 
        
    Output:
         a matrix with wavelength, bandwidth, intensity recieved followed the input sequence

        
    '''
    # preparation
    filter_number = len(opti_wavelength)
    N,M,Q = hypercube.shape
    hypercube = hypercube.reshape(N*M,Q)
    Intensity_recieved = zeros((N*M,Q))
    #number_of_filter 
    L = len(opti_wavelength)
    filters = []
    
    #matrxi for wavelength and bw, intensity, ranked by intensity
    matrix = zeros((L,3))
    matrix[:,0] = opti_wavelength
    matrix[:,1] = opti_bw

    # Define filter responses, add them together, not a elegant way, should improve it later
    filters = [exp(-(wavelength - opti_wavelength[i])**2 / (2*(opti_bw[i]/2.355)**2)) for i in range(len(opti_wavelength))]
       
    
 
    #print(hypercube[1].shape)
    #print(filters[0].shape)
    #plot(wavelength,filters[0],color= 'blue')
    
    #print(np.dot(hypercube[1],filters[1]))
    
    #intensity factor
    for m in range(0,L):
        for n in range(0,N*M):
            Intensity_recieved[n,m]=np.dot(hypercube[n],filters[m])
        sum_over_spectra = Intensity_recieved.sum(1)
        
        sum_over_pixel = sum_over_spectra.sum()
        matrix[m,2] = sum_over_pixel


    return matrix

unmixing_weighted_accuracy/. haven't implemented correctly

In [4]:
def compute_unmixing_weighted_accuracy(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q

            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)
    
    ranked = intensity_recieved(hypercube,bandwidths, center_wavelengths,wavelength, verbose=False)
    weight_sq_before_normalized = (ranked[:,2])
    weight = sqrt(weight_sq_before_normalized/sum(weight_sq_before_normalized))

    # Take RMS difference between true abundance map and computed.
    difference = abundance_map.reshape((N*M,len(endmember_sig)))-predicted
    
    for k in range(0,L):
        weight_difference = difference[:,k]* weight[k] 
        
    
    accuracy = sqrt(mean((weight_difference)**2))
    
    if not accuracy or isnan(accuracy):
        return 100.
    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

# 1.1.2. Hypercubereconstruction

RMSE

In [5]:
def compute_hypercube_reconstruction_accuracy_RMSE(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q
            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)
    
    predicted = predicted.reshape(N,M,P)

    pred_hypercube,_ = create_sythentic_hypercube(predicted, endmembers, wavelength)
    
    # Take RMS difference between true abundance map and computed.
    accuracy = sqrt(mean((pred_hypercube-hypercube)**2))
    
    if not accuracy or isnan(accuracy):
        return 100.
    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

DFT

In [6]:
from scipy.fft import fft, ifft
import numpy as np

def mag_dft (x):
    m = x.shape[0]
    magti = x
    for i in range (m):
        real_x = np.real(x[i])
        imag_x = np.imag(x[i])
        magti[i] = np.sqrt(np.mean(real_x**2+imag_x**2))
    return magti

def compute_hypercube_reconstruction_accuracy_DFT(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q
            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)
    
    predicted = predicted.reshape(N,M,P)

    pred_hypercube,_ = create_sythentic_hypercube(predicted, endmembers, wavelength)
    
    xf = fft(hypercube)
    yf = fft(pred_hypercube)
    
    xm = mag_dft (xf)
    ym = mag_dft (yf)
    
    # Take real part RMS difference between true abundance map and computed in DFT  space.
    accuracy = sqrt(mean((ym-xm)**2))
    
    
    if not accuracy or isnan(accuracy):
        return 100.
    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

SAM

In [7]:
from  pysptools import distance

def compute_hypercube_reconstruction_accuracy_SAM(hypercube, abundance_map, endmembers, wavelength, center_wavelengths, bandwidths, return_predicted=False):
    '''
    Function to compute unmixing accuracy using NNLS spectral unmixing given a set of spectral band 
    center wavelengths and bandwidths. Assuming gaussian filter responses with a bandwidth defined
    as the FWHM.
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        abundance_map (N x M x P)- 3D ground-truth abundance map of target. Each channel in L corresponds to the
             abundance (between 0 and 1) of the spectral signature in the spectra array. 
        
        endmembers (P x Q array) - Array of endmember signals to unmix.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.
        
        center_wavelengths (1 x L array) - Array or list of center wavelengths for Gaussian filter responses
            to use for unmixing.
        
        bandwidths (1 x L array) - Array or list of bandwidths for Gaussian filter responses
            to use for unmixing.
        
    Outputs:
        accuracy (float) - RMS error of the unmixed abundance compared to the gruond truth
        
        predicted (optional N x M x P array) - Predicted abundance map of hypercube
    '''
    # Extract relevant dimensions
    N,M,Q = hypercube.shape
    
    # Define filter responses
    filters = [exp(-(wavelength - center_wavelengths[i])**2 / (2*(bandwidths[i]/2.355)**2)) for i in range(len(center_wavelengths))]
    
    P = len(endmembers)
    L = len(filters)
    
    # Pre-allocate arrays
    data_sig = zeros((N*M,L))
    endmember_sig = zeros((P,L))
    
    # Iterate through each filter
    for i,f in enumerate(filters):
        # Compute the signal from each filter received from the data
        # and that received from an endmember.
        data_sig[:,i] = sum(hypercube.reshape((N*M,Q))*f,axis=1) # (N*M) x L
        
        for j, e in enumerate(endmembers):
            endmember_sig[j,i] = sum(endmembers[j]*f) # P x Q
            
    # Reshape the data array into 1d array
    predicted = abundance_maps.amaps.NNLS(data_sig, endmember_sig) # (N*M x P)
    
    predicted = predicted.reshape(N,M,P)

    pred_hypercube,_ = create_sythentic_hypercube(predicted, endmembers, wavelength)

    map_sam = zeros((N,M))
    for i in range(N):
        for j in range (M):
            y_true = pred_hypercube[i,j,:]
            y_pred = hypercube[i,j,:]
            map_sam[i,j] = distance.SAM(y_true,y_pred)
    
    # Take overall SAM.
    accuracy = mean(map_sam)
    
    if not accuracy or isnan(accuracy):
        return 100.
    
    if return_predicted:
        return accuracy, predicted.reshape((N,M,P))
    else:
        return accuracy

# 1.2. Merit functions in spatial optimisation

RGB

In [8]:
class ColourSystem:
    """A class representing a colour system.
    
    Assume pure white, illuminant E

    A colour system defined by the CIE x, y and z=1-x-y coordinates of
    its three primary illuminants and its "white point".

    reference: http://www.brucelindbloom.com/index.html?Eqn_RGB_XYZ_Matrix.html

    """
    # The CIE colour matching function for 380 - 780 nm in 5 nm intervals
    cmf = np.loadtxt('cie-cmf.txt', usecols=(1,2,3))
    wv = arange(380,785,5)
                         
    def __init__(self, wavelength):
        """Initialise the ColourSystem object.

        Pass vectors (ie NumPy arrays of shape (3,)) for each of the
        red, green, blue  chromaticities and the white illuminant
        defining the colour system.
        
        If wavelength is provided, then the CMF function will be re-scaled
        to be on the same sampling as the provided array.

        """

        # Chromaticities
        # The chromaticity matrix (RGB -> XYZ) and its inverse (XYZ -> RGB)
        # Illuminant 'E'
        self.M = array([[0.4887180,0.3106803,0.2006017],[0.1762044,0.8129847,0.0108109],[0.0000000,  0.0102048, 0.9897952]])
        self.MI = np.linalg.inv(self.M)

        self.cmf = array([interp(wavelength, self.wv, self.cmf[:,0]),
                          interp(wavelength, self.wv, self.cmf[:,1]),
                          interp(wavelength, self.wv, self.cmf[:,2])]).T
        
        self.wv = wavelength        
        self.N = np.sum(self.cmf[:,1] * ones((len(self.wv),)))

    def xyz_to_rgb(self, xyz):
        """
        Transform from xyz to rgb representation of colour.
        """
        rgb = self.MI.dot(xyz)
        
        # We're not in the RGB gamut: approximate by desaturating
        rgb[rgb<0] = 0
        rgb[rgb>1] = 1

        return rgb

    def spec_to_xyz(self, spec):
        """
        Convert a spectrum to an xyz point.
        """
        XYZ = np.sum(spec[:, np.newaxis] * self.cmf, axis=0)
        return XYZ / self.N

    def spec_to_rgb(self, spec):
        """Convert a spectrum to an rgb value."""
        xyz = self.spec_to_xyz(spec)
        return self.xyz_to_rgb(xyz)

NameError: name 'arange' is not defined

In [9]:
def MSFA_to_RGB(MSFA, filters, wavelength, verbose=False):
    '''
    Convert demosaicked MSFA data to RGB spectra using filter responses to
    represent complete spectra.
    
    Inputs:
    
        MSFA (N x M x J) - MSFA demosaiced data array with J channels / filters. 
        
        filters (J x Q) - Normalized (max of 1) spectral response curves, or QE curves, for each 
            filter in the MSFA. A total of J different filter responses of the macropixel
            should be provided in reading order. J will equal the number of sub-pixels in the macro-pixel.
            e.g., px*py = J. Even if there is a duplicate filter, provide the same spectral response curve.
        
        wavelength (Q x 1) - Array describing the wavelength value corresponding to the spectral
            dimension of the hypercube. Filter spectral responses must also be provided on this axis.
        
        cs - ColorSystem object defining RGB colorspace
        
        verbose (optional) - Set to true for status updates.
    
    Outputs: 
        MSFA_RGB (N x M x 3 array) - RGB-equivalent image generated from MSFA data
    
    '''
    px,py,channels = MSFA.shape
    
    print('channel shape',MSFA.shape)
    print('filter shape',filters.shape)
    
    cs = ColourSystem(wavelength)
    
    MSFA_RGB = zeros((px,py,3))
    
    # Can we do this with an array operation to speed up the 
    # processing?
    
    for j in range(py):
        if verbose and not j % 100:
            print("Converting row %d" %j)
        for i in range(px):
            spectra = zeros_like(filters[0])
            spectra = spectra.astype(np.float64)

            for c in range(0,int(channels-1)):
                #print('c',c)
                spectra += MSFA[j,i,c]*filters[c]
                
            MSFA_RGB[j,i,:] = cs.spec_to_rgb(spectra)
            
    return MSFA_RGB

In [10]:
def hypercube_to_RGB(hypercube,wavelength,verbose=False):
    '''
    Convert hyperspectral data to RGB
    
    Inputs:
        hypercube (N X M x Q) - 3D synthetic hypercube.
        
        wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
            response arrays.    
            
        cs - ColorSystem object defining RGB colorspace
        
    Outputs:
        hypercube_RGB (N x M x 3) - RGB-equivalent image generated from hyperspectral data.
    
    '''
    px,py,wv = hypercube.shape
    
    hypercube_RGB = zeros((px,py,3))
    
    cs = ColourSystem(wavelength)
    
    # Can we do this with an array operation to speed up the 
    # processing?
    
    for j in range(py):
        if verbose and not j % 100:
            print("Converting row %d" %j)
        for i in range(px):
            hypercube_RGB[j,i,:] = cs.spec_to_rgb(hypercube[j,i])
            
    return hypercube_RGB

In [11]:
def RMS_pixel_difference(im,ref):
    '''
    Total RMS error between an image (im) and reference (ref). Used as a merit function.
    
    Inputs:
        im (N x M array) - Image to test quality.
    
        ref (M x M array) - Reference to compare test image to.
    
    Outputs:
    
        rms (float): RMS pixel difference between input image and a reference.
    
    '''
    
    # How to account for scaling between channels?
    d = im / im.max(axis=(0,1))
    rms = sqrt( mean((d-ref)**2) )
        
    return rms

join spectral-spatial 

In [12]:
def spatial_optimization_MSFA(hypercube,wavelength,pattern_dims, filters, verbose=False):
        '''
        Full spatial optimization pipeline to determine the optimal filter
        pattern for a multispectral filter array given a synthetic hypercube
        target, and filter responses.
        
        Inputs:
            hypercube (N X M x Q) - 3D synthetic hypercube.
            
            wavelength (1 x Q array) - Array describing the wavelength value corresponding to the spectral
                response arrays.
            
            pattern_dims ((my, mx)) - Tuple of dimensions for macropixel. A 3x3 pixel would be
                provided as (3,3) whereas a 3x2 pixel is provided as (3,2), etc.

            filters (J x Q) - Normalized (max of 1) spectral response curves, or QE curves, for each 
                filter in the MSFA. A total of J different filter responses of the macropixel
                should be provided in reading order. J will equal the number of sub-pixels in the macro-pixel.
                e.g., px*py = J. Even if there is a duplicate filter, provide the same spectral response curve.
            
            verbose (optional): Set to true for status updates
            
        Outputs:
            pattern for the min unmixing error
            demosaicked images (without duplicated bands) for the min unmixing error
            demosaicked images (with duplicated bands) for the min unmixing error
            reference images for each band
            a list of unmixing error for each spatial arrangement
            a list of demosaicking rms error for each spatial arrangement
        '''
        # Total number of filters in the pattern
        N_filters = pattern_dims[1]*pattern_dims[0]#len(filters)
        # Available filters to place
        avail_fils = arange(len(filters))
        # pattern shape
        my,mx = pattern_dims
        
        # Iterate through every possible permutation of the filter arrangement
        best = inf
        pattern = []
        best_demosaicked = None
        best_demosaicked_MSI = None
        best_ref = None
        demo_err_list =[]
        accu_list = []

        filter_combos = all_filters_present(avail_fils,N_filters)
        
        for fil_set in filter_combos:
            if verbose == True:
                print(fil_set)
            
            # Re-organize filters appropriately
            f = array([filters[i] for i in fil_set])
            
            # Generate raw MSFA data
            raw, _ = sample_hypercube_MSFA(hypercube,pattern_dims,f,wavelength,full_res=False)

            # Generate perfect spatial reference
            ref,_ = sample_hypercube_MSFA(hypercube,pattern_dims,filters,wavelength,full_res=True)
   
            # Demosaic the raw MSFA data using bilinear
            H = compute_H(*pattern_dims)
            demosaicked = zeros_like(raw)
            demosaicked_MSI = zeros_like(ref) # in the order of given filter set
            MSI_count = zeros(len(filters))

            
            for c in range(len(f)):
                demosaicked[:,:,c] = WB(raw[:,:,c],H)
                demosaicked_MSI[:,:,fil_set[c]] = demosaicked_MSI[:,:,fil_set[c]] + demosaicked[:,:,c]
                MSI_count[fil_set[c]] += 1
                
            # average the duplicated bands
            demosaicked_MSI = demosaicked_MSI / MSI_count[newaxis, newaxis, :]
            
            
            # apply mask
            for c in range(len(f)):
                i = int(c / my) # in x direction
                j = c - i*my # in y direction
                demosaicked_MSI[j::my,i::mx,fil_set[c]] = ref[j::my,i::mx,fil_set[c]]

            m = RMS_pixel_difference(demosaicked_MSI, ref)
            # note that the edges of an image may not be well demosaicked
            accu = MSI_criteria(demosaicked_MSI[5:-5,5:-5,:], a_map[5:-5,5:-5,:], spectra, wavelength, filters, 'NNLS', return_predicted=False)
            demo_err_list.append(m)
            accu_list.append(accu)
            
            if verbose == True:
                print(m, accu)
                
            # the best pattern should give best unmixing accuracy
            if accu < best:
                best = accu
                pattern = fil_set
                best_demosaicked = demosaicked
                best_demosaicked_MSI = demosaicked_MSI
                best_ref = ref

        print("Optimal pattern is", pattern)
        return pattern,  demo_err_list, accu_list

# Duplications 

In [13]:
def intensity_rank(hypercube,opti_bw, opti_wavelength,t = 1., wavelength, verbose=False):
    
    '''
    Inputs:
        1) hypercube (N X M x Q) - 3D synthetic hypercube.
        2) bandwidth
        3) wavelength
        4) transmissivity 
  
    
        ranking depends on three factor: 
        1) Total area of each filter 
        2) The total intensity recieved by this chanel 
        
    Output:
         a matrix with wavelength, bandwidth, intensity recieved
         
    Could be improve:
       ranking way
        
    '''
    # preparation
    filter_number = len(opti_wavelength)
    N,M,Q = hypercube.shape
    hypercube = hypercube.reshape(N*M,Q)
    print('hypercube shape',hypercube.shape)
    Intensity_recieved = zeros((N*M,Q))
    #number_of_filter 
    
    L = len(opti_wavelength)
    print(L)
    filters = []
    
    #matrxi for wavelength and bw, intensity, ranked by intensity
    matrix = zeros((L,3))
    matrix[:,0] = np.array(opti_wavelength)
    matrix[:,1] = np.array(opti_bw)

    # Define filter responses, add them together, not a elegant way, should improve it later
    filters = t * [exp(-(wavelength - opti_wavelength[i])**2 / (2*(opti_bw[i]/2.355)**2)) for i in range(len(opti_wavelength))]
    
    print('number of filter type',len(filters))    
    
    
    #intensity factor
    for m in range(0,L):
        for n in range(0,N*M):
            Intensity_recieved[n,m]=np.dot(hypercube[n],filters[m])
        sum_over_spectra = Intensity_recieved.sum(1)
        
        sum_over_pixel = sum_over_spectra.sum()
        matrix[m,2] = sum_over_pixel
    
    #rank it by intensity, from low to high
    matrix_ranked = matrix[np.argsort(matrix[:,2])]

    return matrix_ranked

SyntaxError: non-default argument follows default argument (<ipython-input-13-7f4627601e65>, line 1)

In [14]:
def generate_duplication(pattern_dims,hypercube,opti_bw, opti_wavelength,wavelength, verbose=False):
    
    '''
    Inputs:
        1) hypercube (N X M x Q) - 3D synthetic hypercube.
        2) bandwidth
        3) wavelength
        4) number of block (mxXny)
        
    Output:
         f_filter that matched with number of block in the pattern (blocks, wavlength region)
         
    Could be improve:
       ranking way
    '''
    ranked_table = intensity_rank(hypercube,opti_bw, opti_wavelength,wavelength, verbose=False)
    filter_number,_ = ranked_table.shape
    print('filter_number',filter_number)

    X,Y = pattern_dims
    number_of_block = X*Y
    print('number_of_block',number_of_block)
    
    W = wavelength.shape[0]
    f = zeros((number_of_block,int(W)))
    filter_labels = zeros((number_of_block))
    
    if filter_number > number_of_block:
        # use top intensity 
        for i in range(0,(number_of_block)):
            print(i)
            # pick from high intensity
            number = filter_number-1-i
            print(number)
            opti_wavelength = ranked_table[number,0]
            opti_bw = ranked_table[number,1]
            print(opti_wavelength)
            f[i] = exp(-(wavelength - opti_wavelength)**2 / (2*(opti_bw/2.355)**2))
            filter_labels[i] = opti_wavelength

    else:
        # need duplicate
        for i in range(0,(number_of_block)):
            # pick from high intensity
            number = filter_number-i-1
            if number >= 0:
                number = number
                opti_wavelength = ranked_table[number,0]
                opti_bw = ranked_table[number,1]
                f[i] = exp(-(wavelength - opti_wavelength)**2 / (2*(opti_bw/2.355)**2))
                filter_labels[i] = opti_wavelength
                print('No',number)
                print('opti_wave',opti_wavelength)
            else:
                number = filter_number - abs(number)
                opti_wavelength = ranked_table[number,0]
                opti_bw = ranked_table[number,1]
                f[i] = exp(-(wavelength - opti_wavelength)**2 / (2*(opti_bw/2.355)**2))
                filter_labels[i] = opti_wavelength
                print('No',number)
                print('opti_wave',opti_wavelength)
            #print('filter',number)
    #print("filters respond shape:", f.shape)

        
    return f, filter_labels